## 利用pulp模块解决运输问题

在之前负责的风电项目叶片发运过程中，间或涉及到运输问题。恰好最近接触了一点优化方面的知识，对叶片的发运问题做一个优化。

当前项目为一个100MW的风电电站，场地位于B国境内。设计拟选用2MW的风机50台，因此需运输叶片50套（单车可1套3片）。  
受到生产日期和船期限制，叶片分别发至B国的1,2两个码头，1码头20套，2码头30套，并由卡车运输项目地。  
综合勘察后，选定施工地点5处，分别为1,2,3,4,5。各地需要的叶片套数分别为5,6,11,13,15。  
考虑到运输距离及路况，1码头向1-5施工地运输单套叶片的费用（美元）分别为：100,200,220,150,130；  
2码头向1-5施工地运输单套叶片的费用分别为：80,210,180,120,160；

综合以上信息，提供一个优化的运输方案。

### 1.构建数据集

In [1]:
#导入线性规划包pulp
from pulp import *

In [2]:
#创建一个关于码头的list

ports=[1,2]

#创建一个1,2两码头供应叶片的dict

supply={1:20,2:30}

#创建一个需求节点的dict

sites={'1':5,'2':6,'3':11,'4':13,'5':15}

In [3]:
#创建一个1,2两码头向各场地运输叶片运费的dict

costs = {
        1:{'1':100,'2':200,'3':220,'4':150,'5':130},
        2:{'1':80,'2':210,'3':180,'4':120,'5':160}
        }

### 2.分析问题与构建模型

#### 2.1明确问题

In [4]:
'''
构建问题
显然本问题旨在优化运输路径，使得运费最小，因此是一个最小化的问题
'''

prob=LpProblem('Blades transportation',LpMinimize)

In [5]:
'''
构建所有的运输路径
'''

routes=[(i,j) for i in ports for j in sites]
routes

[(1, '1'),
 (1, '2'),
 (1, '3'),
 (1, '4'),
 (1, '5'),
 (2, '1'),
 (2, '2'),
 (2, '3'),
 (2, '4'),
 (2, '5')]

In [6]:
'''
构建关于路径的变量
由于运输的叶片都是整套，因此这里是个整数问题
'''
route_vars=LpVariable.dicts('Route',(ports,sites),0,None,LpInteger)
route_vars

{1: {'1': Route_1_1,
  '2': Route_1_2,
  '3': Route_1_3,
  '4': Route_1_4,
  '5': Route_1_5},
 2: {'1': Route_2_1,
  '2': Route_2_2,
  '3': Route_2_3,
  '4': Route_2_4,
  '5': Route_2_5}}

#### 2.2添加目标函数与约束

In [7]:
'''
构建目标函数—总运输成本
'''

prob += lpSum([route_vars[i][j]*costs[i][j] for (i,j) in routes])

In [8]:
'''
增加约束条件
'''

#对码头而言，向5个场地供应的叶片数不应大于其叶片数量
for i in ports:
    prob += (lpSum([route_vars[i][j] for j in sites]) <= supply[i])

#对施工场地而言，每个场地的供应数应等于其需求数
for j in sites:
    prob +=(lpSum([route_vars[i][j] for i in ports])==sites[j])    

上限约束问题:  
显然，本次分析仅对各路径运输量之和进行了约束，即每次运送叶片数量>=0;  
而有的时候，可能会对每条运输路径进行上下限的约束。  
如此，则有如下形式：  
 Route_1_1 =LpVariable("Route_1_1", 0, 1)

系数不为1情况：  
本次约束条件各变量系数均为1。  
但一般问题中，我们必然会遇到大量约束，且系数不为1的情况，这种情况可以利用pulp包中的LpDot函数，将变量和系数矩阵相乘计算，如下：  
prob += (pulp.lpDot(route_vars, const_mat)=< upbound)

### 3.计算结果

In [9]:
'''
计算优化的分配结果
solve命令可以查看当前问题的方程式；
solve()可以进行求解
'''
prob.solve

<bound method LpProblem.solve of Blades transportation:
MINIMIZE
100*Route_1_1 + 200*Route_1_2 + 220*Route_1_3 + 150*Route_1_4 + 130*Route_1_5 + 80*Route_2_1 + 210*Route_2_2 + 180*Route_2_3 + 120*Route_2_4 + 160*Route_2_5 + 0
SUBJECT TO
_C1: Route_1_1 + Route_1_2 + Route_1_3 + Route_1_4 + Route_1_5 <= 20

_C2: Route_2_1 + Route_2_2 + Route_2_3 + Route_2_4 + Route_2_5 <= 30

_C3: Route_1_1 + Route_2_1 = 5

_C4: Route_1_2 + Route_2_2 = 6

_C5: Route_1_3 + Route_2_3 = 11

_C6: Route_1_4 + Route_2_4 = 13

_C7: Route_1_5 + Route_2_5 = 15

VARIABLES
0 <= Route_1_1 Integer
0 <= Route_1_2 Integer
0 <= Route_1_3 Integer
0 <= Route_1_4 Integer
0 <= Route_1_5 Integer
0 <= Route_2_1 Integer
0 <= Route_2_2 Integer
0 <= Route_2_3 Integer
0 <= Route_2_4 Integer
0 <= Route_2_5 Integer
>

In [10]:
print('objective',pulp.value(prob.objective))

objective None


In [11]:
for (i,j) in routes:
    a=pulp.value(route_vars[i][j])
    b=route_vars[i][j]
    print('%s的值是%s'%(b,a))

Route_1_1的值是None
Route_1_2的值是None
Route_1_3的值是None
Route_1_4的值是None
Route_1_5的值是None
Route_2_1的值是None
Route_2_2的值是None
Route_2_3的值是None
Route_2_4的值是None
Route_2_5的值是None


参考链接：https://www.coin-or.org/PuLP/CaseStudies/a_transportation_problem.html